In [1]:
from choice_model import ChoiceModel

folder = 'new_graph'

def predict(period,remove_type='Long-term Project'):
    model = ChoiceModel()
    prev_period_file = f'{folder}/{folder}_{period-1}.pkl'
    new_model = ChoiceModel(graph_path=prev_period_file) 
    graph = model.get_period(period)
    new_nodes = [n for n in graph.nodes if n not in list(new_model.graph.nodes)]
    new_nodes = [n for n in new_nodes if graph.nodes[n]['type'] != remove_type]
    test_graph = new_model.graph.copy()
    test_graph = model.predict_links(test_graph, new_nodes, file_name=f"{folder}/{folder}_{period}.pkl",
                                     choice_type=['Actors'])
    new_model = ChoiceModel(graph_path=f"{folder}/{folder}_{period}.pkl")
    issolated_nodes = new_model.get_isolated_nodes()
    if len(issolated_nodes) > 0:
        test_graph = new_model.graph.copy()
        test_graph = model.predict_links(test_graph, issolated_nodes,file_name=f"{folder}/{folder}_{period}.pkl",k1=10,k2=5,top_k=5)
        new_model.remove_isolated_nodes()
    new_model.visualize(file_name=f"{folder}/{folder}_{period}.html")
    PPI, CVI, DCI = new_model.evaluate(test_graph)
    print(f"Period {period} PPI: {PPI} CVI: {CVI} DCI: {DCI}")


In [2]:
for i in range(1,5):
    predict(i)

  1%|▏         | 1/75 [00:07<09:28,  7.69s/it]

'answer'


 56%|█████▌    | 42/75 [08:49<07:15, 13.19s/it]

'answer'


 77%|███████▋  | 58/75 [19:56<16:02, 56.61s/it] 

'answer'


100%|██████████| 75/75 [23:12<00:00, 18.57s/it]


Period 1 PPI: 0.3486439580827335 CVI: 4.520408163265306 DCI: 0.07692307692307693


  8%|▊         | 9/118 [01:46<19:10, 10.56s/it]